In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

from keras import layers, models

from finrock.data_feeder import PdDataFeeder
from finrock.trading_env import TradingEnv
from finrock.scalers import MinMaxScaler, ZScoreScaler
from finrock.reward import SimpleReward, AccountValueChangeReward
from finrock.metrics import DifferentActions, AccountValue, MaxDrawdown, SharpeRatio
from finrock.indicators import BolingerBands, RSI, PSAR, SMA, MACD

from rockrl.utils.misc import MeanAverage
from rockrl.utils.memory import MemoryManager
from rockrl.tensorflow import PPOAgent
from rockrl.utils.vectorizedEnv import VectorizedEnv

# The following allows to save plots in SVG format.
import matplotlib_inline
%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

ModuleNotFoundError: No module named 'rockrl'

In [2]:
df = pd.read_csv('Datasets/random_sinusoid.csv')
df = df[:-1000]


pd_data_feeder = PdDataFeeder(
    df,
    indicators = [
        BolingerBands(data=df, period=20, std=2),
        RSI(data=df, period=14),
        PSAR(data=df),
        MACD(data=df),
        SMA(data=df, period=7),
    ]
)

num_envs = 10
env = VectorizedEnv(
    env_object = TradingEnv,
    num_envs = num_envs,
    data_feeder = pd_data_feeder,
    output_transformer = ZScoreScaler(),
    initial_balance = 1000.0,
    max_episode_steps = 1000,
    window_size = 50,
    reward_function = AccountValueChangeReward(),
    metrics = [
        DifferentActions(),
        AccountValue(),
        MaxDrawdown(),
        SharpeRatio(),
    ]
)

action_space = env.action_space
input_shape = env.observation_space.shape

def actor_model(input_shape, action_space):
    input = layers.Input(shape=input_shape, dtype=tf.float32)
    x = layers.Flatten()(input)
    x = layers.Dense(512, activation='elu')(x)
    x = layers.Dense(256, activation='elu')(x)
    x = layers.Dense(64, activation='elu')(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(action_space, activation='softmax')(x) # discrete action space
    return models.Model(inputs=input, outputs=output)

def critic_model(input_shape):
    input = layers.Input(shape=input_shape, dtype=tf.float32)
    x = layers.Flatten()(input)
    x = layers.Dense(512, activation='elu')(x)
    x = layers.Dense(256, activation='elu')(x)
    x = layers.Dense(64, activation='elu')(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(1, activation=None)(x)
    return models.Model(inputs=input, outputs=output)

agent = PPOAgent(
    actor = actor_model(input_shape, action_space),
    critic = critic_model(input_shape),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    batch_size=128,
    lamda=0.95,
    kl_coeff=0.5,
    c2=0.01,
    writer_comment='ppo_sinusoid_discrete',
)

pd_data_feeder.save_config(agent.logdir)
env.env.save_config(agent.logdir)

memory = MemoryManager(num_envs=num_envs)
meanAverage = MeanAverage(best_mean_score_episode=1000)
states, infos = env.reset()
rewards = 0.0
while True:
    action, prob = agent.act(states)

    next_states, reward, terminated, truncated, infos = env.step(action)
    memory.append(states, action, reward, prob, terminated, truncated, next_states, infos)
    states = next_states

    for index in memory.done_indices():
        env_memory = memory[index]
        history = agent.train(env_memory)
        mean_reward = meanAverage(np.sum(env_memory.rewards))

        if meanAverage.is_best(agent.epoch):
            agent.save_models('ppo_sinusoid')

        if history['kl_div'] > 0.05 and agent.epoch > 1000:
            agent.reduce_learning_rate(0.995, verbose=False)

        info = env_memory.infos[-1]
        print(agent.epoch, np.sum(env_memory.rewards), mean_reward, info["metrics"]['account_value'], history['kl_div'])
        agent.log_to_writer(info['metrics'])
        states[index], infos[index] = env.reset(index=index)

    if agent.epoch >= 10000:
        break

env.close()


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
1007.7134780389868 0.6123856339139063
1090.5607599465382 0.5834140165388254
846.6079315479225 0.3566458368008232
365.1254298103569 -0.37882083613511625
1915.0183999296175 1.2448783426793206
1354.2931805116207 0.15978317389131486
733.5148192828475 0.2843296256162624
1128.2484363607632 0.7170146311645123
2419.7279037625744 0.5996566291942848
2322.4871577574663 0.6823150472780253
864.3927569584265 0.42316124987873255
236.87971702222544 -0.9249763557178783
681.5403164544242 0.1297951333098099
1089.7544202141632 0.531591718689451
595.1160152777511 -0.005048374172072281
2065.4706098426273 0.31158160419666364
6874.459650816223 1.1661137526697425
1146.4553799780242 0.5980774592489487
1918.9167765851373 -0.11130718481958958
2210.257708239037 0.7926537560861525
651.3874451898511 -0.47676957364820255
1788.3558763363394 0.4295761584173775
2050.430960591166 1.2678191173795266
4400.843415

KeyboardInterrupt: 